Estatísticas Gerais 1

In [1]:
import pandas as pd
import random
import numpy as np


In [2]:
data = pd.read_csv('credit_data.csv')

In [3]:
data.shape


(2000, 5)

In [4]:
data.head()

,i#clientid,income,age,loan,c#default
0,1,66155.925095,59.017015,8106.532131,0
1,2,34415.153966,48.117153,6564.745018,0
2,3,57317.170063,63.108049,8020.953296,0
3,4,42709.534201,45.751972,6103.642260,0
4,5,66952.688845,18.584336,8770.099235,1


In [5]:
data.isnull().sum()

i#clientid    0
income        0
age           3
loan          0
c#default     0
dtype: int64

In [6]:
data.dropna(inplace=True)

In [7]:
data.isnull().sum()

i#clientid    0
income        0
age           0
loan          0
c#default     0
dtype: int64

In [8]:
data.shape

(1997, 5)

**Amostragem aleatória simples**

In [9]:
#função amostragem aleatória simples

def amostragem_aleatoria_simples2(dataset, amostras, random_state=1):
    return dataset.sample(n=amostras, random_state=random_state)

In [10]:
amostragem_aleatoria_simples = amostragem_aleatoria_simples2(data, 1000)
amostragem_aleatoria_simples.shape

(1000, 5)

**Amostragem Sistemática**

In [11]:
# Função amostra sistemática

def amostragem_sistematica2(dataset, amostras):
    intervalo = len(dataset) // amostras
    random.seed(42)
    inicio = random.randint(0, intervalo)
    indices = np.arange(inicio, len(dataset), step=intervalo)
    indices = indices[:amostras]
    amostra_sistematica = dataset.iloc[indices]
    return amostra_sistematica

In [12]:
amostragem_sistematica = amostragem_sistematica2(data, 1000)
amostragem_sistematica.shape

(1000, 5)

**Amostragem por Agrupamento**

In [13]:
#Amostragem por agrupamento

import random

def amostragem_agrupamento2(dataset, numero_grupos):
    intervalo = len(dataset) // numero_grupos
    grupos = []
    
    id_grupo = 0
    contagem = 0
    
    for _ in dataset.iterrows():
        grupos.append(id_grupo)
        contagem += 1
        if contagem >= intervalo:
            contagem = 0
            id_grupo += 1
            
    dataset['grupo'] = grupos
    grupo_selecionado = random.randint(0, numero_grupos -1)
        
    return dataset[dataset['grupo'] == grupo_selecionado]

In [14]:
amostragem_agrupamento = amostragem_agrupamento2(data, 10)
amostragem_agrupamento.shape

(199, 6)

**Amostragem Reservatório**

In [15]:
import random

def amostragem_reservatorio2(dataset, amostras):
    tamanho = len(dataset)
    reservatorio = []

    # Inicializa o reservatório com os primeiros 'amostras' índices
    for i in range(amostras):
        reservatorio.append(i)

    # Amostragem de reservatório para o restante dos itens
    for i in range(amostras, tamanho):
        j = random.randrange(i + 1)
        if j < amostras:
            reservatorio[j] = i

    # Retorna as linhas do DataFrame correspondentes aos índices no reservatório
    return dataset.iloc[reservatorio]


In [16]:
amostragem_reservatorio = amostragem_reservatorio2(data, 1000)
amostragem_reservatorio.shape

(1000, 6)

**Amostragem Estratificada**

In [17]:
#Encontrando a porcentagem de 1000 entradas em relação ao DF
1000 / len(data)

0.5007511266900351

In [18]:
data['c#default'].value_counts()

0    1714
1     283
Name: c#default, dtype: int64

In [21]:
from sklearn.model_selection import StratifiedShuffleSplit

In [22]:
def amostragem_estratificada2(dataset, percentual): 

    split = StratifiedShuffleSplit(test_size=percentual, random_state=1)
    for _, y in split.split(dataset, dataset['c#default']):
        df_y = dataset.iloc[y]
    return df_y

In [23]:
amostragem_estratificada = amostragem_estratificada2(data,0.5007511266900351 )
amostragem_estratificada.shape

(1000, 6)

**Comparando as médias de age, income e loan entre todas as amostragens e o DF original**


In [24]:
medias_df_origial = data[['age', 'income', 'loan']].mean()
medias_df_origial


age          40.807559
income    45333.864334
loan       4445.487716
dtype: float64

In [25]:
medias_amostragem_aleatoria_simples = amostragem_aleatoria_simples[['age', 'income', 'loan']].mean()
medias_amostragem_aleatoria_simples

age          40.538799
income    45564.054622
loan       4293.552550
dtype: float64

In [26]:
medias_amostragem_sistematica = amostragem_sistematica[['age', 'income', 'loan']].mean()
medias_amostragem_sistematica

age          41.049792
income    44885.712247
loan       4390.351657
dtype: float64

In [27]:
medias_amostragem_agrupamento =amostragem_agrupamento[['age', 'income', 'loan']].mean()
medias_amostragem_agrupamento

age          39.836889
income    44121.995063
loan       4358.447221
dtype: float64

In [28]:
medias_amostragem_estratificada = amostragem_estratificada[['age', 'income', 'loan']].mean()
medias_amostragem_estratificada

age          41.063346
income    45577.508590
loan       4507.002790
dtype: float64

**Pudemos observar que em todos os resultados obtivemos aproximações maiores em uma ou mais variáveis dependendo do tipo de amostra. É importante destacar que é esperado que nenhum algoritmo é de capturar uma amostragem com precisão de 100%, visto que estamos lidando com aproximações. Portanto, devemos nos atentar ao objetivo do negócio para escolher a amostragem mais apropriada. Por exemplo: se quisermos uma amostragem que represente a idade média com a melhor aproximação, a Amostragem Aleatória Simples parece mais apropriada com a média de 40.538799. Por outro lado, se pretendemos ter maior precisão em loan (empréstimo) a Amostragem Sistemática parece a melhor escolha com a média de loan em 4390.351657**